<a href="https://colab.research.google.com/github/okuehnd/NYTArticleVisualization/blob/main/THIS_ONE_WORKSD3_InfoVis_FinalProjectKeywordSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import pandas as pd
import bokeh
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Slider, CustomJS, CheckboxGroup, LinearColorMapper, HoverTool, WheelZoomTool, Div, Toggle, RadioGroup,TextInput, Button
from bokeh.transform import linear_cmap
from bokeh.palettes import Viridis256
from datetime import datetime, timedelta
from pyproj import Proj, transform
from bokeh.models import TileSource
from bokeh.io import show
from bokeh.models.tiles import WMTSTileSource
from pyproj import Transformer
from bokeh.io import push_notebook,save
from bokeh.models import PrintfTickFormatter
from IPython.core.display import display, HTML

In [ ]:
!pip install ipyd3

In [ ]:
csv_filename = '/content/drive/MyDrive/Colab Notebooks/InfoVis_FinalProject/InfoVisFinalProject/sentiment_nyt_clean_mini.csv'
original_df = pd.read_csv(csv_filename)

In [ ]:
articles = original_df.copy()

In [ ]:
articles.head(4)

,abstract,web_url,headline,keywords,pub_date,section_name,sentiment,sentiment_score
0,Many experts on Y2K computer problem report th...,https://www.nytimes.com/2000/01/01/us/1-1-00-t...,Internet's Cheering Squad Nervously Watches Clock,"['electronic mail', 'computer software', 'two ...",2000-01-01 05:00:00+00:00,U.S.,1,0.992791
1,Article on Florida State offensive tackle Todd...,https://www.nytimes.com/2000/01/01/sports/coll...,COLLEGE FOOTBALL; From Homeless to a Home at F...,"['florida state university', 'virginia polytec...",2000-01-01 05:00:00+00:00,Sports,1,0.968833
2,Judge D Irvin Couvillion rules that Burton W K...,https://www.nytimes.com/2000/01/01/business/ta...,Tax Lawyer Called Architect of a Tax-Evasion S...,"['travelers insurance co', 'internal revenue s...",2000-01-01 05:00:00+00:00,Business Day,-1,0.959399
3,"Joshua Davis, high school student from Montcla...",https://www.nytimes.com/2000/01/01/nyregion/fo...,Folds of Newspapers Yield Symbols of Peace,"['montclair (nj)', 'davis, joshua', 'newspaper...",2000-01-01 05:00:00+00:00,New York,1,0.538960


In [ ]:
print((pd.to_datetime(articles.loc[0]['pub_date'])).year)

2000


In [ ]:
articles['pub_date'] = (pd.to_datetime(articles['pub_date'])).dt.year
articles.head(4)

,abstract,web_url,headline,keywords,pub_date,section_name,sentiment,sentiment_score
0,Many experts on Y2K computer problem report th...,https://www.nytimes.com/2000/01/01/us/1-1-00-t...,Internet's Cheering Squad Nervously Watches Clock,"['electronic mail', 'computer software', 'two ...",2000,U.S.,1,0.992791
1,Article on Florida State offensive tackle Todd...,https://www.nytimes.com/2000/01/01/sports/coll...,COLLEGE FOOTBALL; From Homeless to a Home at F...,"['florida state university', 'virginia polytec...",2000,Sports,1,0.968833
2,Judge D Irvin Couvillion rules that Burton W K...,https://www.nytimes.com/2000/01/01/business/ta...,Tax Lawyer Called Architect of a Tax-Evasion S...,"['travelers insurance co', 'internal revenue s...",2000,Business Day,-1,0.959399
3,"Joshua Davis, high school student from Montcla...",https://www.nytimes.com/2000/01/01/nyregion/fo...,Folds of Newspapers Yield Symbols of Peace,"['montclair (nj)', 'davis, joshua', 'newspaper...",2000,New York,1,0.538960


In [ ]:
print(type(articles['pub_date'].loc[833001]))

<class 'numpy.int32'>


In [ ]:
print(type(articles['sentiment'].loc[833001]))

<class 'str'>


In [ ]:
articles['sentiment'] = articles['sentiment'].apply(lambda x: "Positive" if x == 1 else "Negative")
articles.head(4)

,abstract,web_url,headline,keywords,pub_date,section_name,sentiment,sentiment_score
0,Many experts on Y2K computer problem report th...,https://www.nytimes.com/2000/01/01/us/1-1-00-t...,Internet's Cheering Squad Nervously Watches Clock,"['electronic mail', 'computer software', 'two ...",2000,U.S.,Positive,0.992791
1,Article on Florida State offensive tackle Todd...,https://www.nytimes.com/2000/01/01/sports/coll...,COLLEGE FOOTBALL; From Homeless to a Home at F...,"['florida state university', 'virginia polytec...",2000,Sports,Positive,0.968833
2,Judge D Irvin Couvillion rules that Burton W K...,https://www.nytimes.com/2000/01/01/business/ta...,Tax Lawyer Called Architect of a Tax-Evasion S...,"['travelers insurance co', 'internal revenue s...",2000,Business Day,Negative,0.959399
3,"Joshua Davis, high school student from Montcla...",https://www.nytimes.com/2000/01/01/nyregion/fo...,Folds of Newspapers Yield Symbols of Peace,"['montclair (nj)', 'davis, joshua', 'newspaper...",2000,New York,Positive,0.538960


In [ ]:
articles = articles.sample(n = 1000, random_state = 42)
articles.head(4)

,abstract,web_url,headline,keywords,pub_date,section_name,sentiment,sentiment_score
1473771,New York’s baseball teams topped the standings...,https://www.nytimes.com/2022/05/01/sports/base...,Shades of ’86: Mets and Yankees Close April on...,"['baseball', 'new york mets', 'new york yankee...",2022,Sports,Positive,0.999730
833001,The Boston Globe received nine Pulitzer Prizes...,https://www.nytimes.com/2011/05/03/business/me...,"William Taylor II, Ex-Publisher of Boston Glob...","['newspapers', 'new york times co', 'boston gl...",2011,Business Day,Positive,0.981372
1069900,"Andrea Lee, 15, who achieved her goal by makin...",https://www.nytimes.com/2014/06/22/sports/golf...,"With Lucy Li Done, Attention Shifts to Another...","[""united states women's open (golf)"", 'golf', ...",2014,Sports,Positive,0.982204
82276,Space shuttle Atlantis is launched from Cape C...,https://www.nytimes.com/2001/02/08/us/shuttle-...,Shuttle Atlantis Heads for Space Station Carry...,"['laboratories and scientific equipment', 'spa...",2001,U.S.,Positive,0.986458


In [ ]:
int_years = articles['pub_date'].unique().tolist()
print(int_years)
int_years.sort()
print(int_years)
sections = articles['section_name'].unique().tolist()

output_notebook()


start_pos = [articles[(articles['pub_date'] == year) & (articles['sentiment'] == "Positive")].shape[0] for year in int_years]
print(start_pos)
start_neg = [articles[(articles['pub_date'] == year) & (articles['sentiment'] == "Negative")].shape[0] for year in int_years]
print(start_neg)
total_start = [a + b for a, b in zip(start_pos, start_neg)]

#assign datasource
data = {
   'abstract' : articles['abstract'],
   'web_url' : articles['web_url'],
   'headline' : articles['headline'],
   'section_name' : articles['section_name'],
   'keywords': articles['keywords'],
   'pub_date': articles['pub_date'],
   'sentiment': articles['sentiment']
}

display_data = {
   'years_pos' : int_years,
   'years_neg': int_years,
   'pos_frequency': start_pos,
   'neg_frequency':  start_neg,
   'total_frequency': total_start,
   'display_freq': total_start
}

source = ColumnDataSource(data)
filtered_source = ColumnDataSource(data.copy())
display_source = ColumnDataSource(display_data)

#create filtering components
year_slider = Slider(start = min(int_years),end = max(int_years),value = min(int_years), step =1, title = 'Year')
checkbox_section = CheckboxGroup(labels = sections, active = list(range(len(sections))))
checkbox_sentiment = CheckboxGroup(labels = ["Positive", "Negative"], active = [0,1])
keyword_search = TextInput(title = 'Search Keyword:')
keyword_display = Div(text="", width=200, height=50)

#temp error logging div
log_div = Div(text="Initial log", width=800, height=100)

#create plot
plot = figure(title = "Keyword Sentiment")
pos_line = plot.line('years_pos','pos_frequency', source = display_source, legend_label = "Positive Sentiment", line_width  =2, color = "green")
pos_scat =plot.scatter('years_pos','pos_frequency', source = display_source, size = 8, color = "green", alpha = 0.6)
neg_line = plot.line('years_neg','neg_frequency', source = display_source, legend_label = "Negative Sentiment", line_width  =2, color = "red")
neg_scat = plot.scatter('years_neg','neg_frequency', source = display_source, size = 8, color = "red", alpha = 0.6)

plot.xaxis.formatter = PrintfTickFormatter(format="%d")

callback = CustomJS(args = dict(source = source,pos_line = pos_line, neg_line = neg_line, neg_scat = neg_scat, pos_scat = pos_scat, display_source = display_source, slider = year_slider, checkbox_section = checkbox_section, checkbox_sentiment = checkbox_sentiment,keyword_search = keyword_search, keyword_display = keyword_display, log_div = log_div, int_years = int_years), code= """

  let display_text = "";

  const data = source.data;
  const original_data = source.original_data || {...data};
  source.original_data = original_data;

  const active_year = slider.value;

  const active_section = checkbox_section.active.map(i => checkbox_section.labels[i]);
  const active_sentiment = checkbox_sentiment.active.map(i => checkbox_sentiment.labels[i]);

  const keyword_value = keyword_search.value ? keyword_search.value.trim().toLowerCase() : "";
  const keywords = keyword_display.text.split(',').map(k => k.trim()).filter(k => k != '');

  if (keyword_value && !keywords.includes(keyword_value)) {
    keywords.push(keyword_value);
  }

  display_text += " HERE3 " + keywords;
  log_div.text = display_text;

  const filtered_abstract = [];
  const filtered_web_url = [];
  const filtered_headline = [];
  const filtered_section_name = [];
  const filtered_keywords = [];
  const filtered_pub_date = [];
  const filtered_sentiment = [];

  const filtered_years = [];
  const filtered_sent = [];

  let yearCounts_pos = {};
  let yearCounts_neg = {};

  // Iterate through the years and x list
  for (let i = 0; i < original_data['abstract'].length; i++) {
    if(original_data['pub_date'][i] <= active_year && active_section.includes(original_data['section_name'][i]) && active_sentiment.includes(original_data['sentiment'][i])) {
      if(keywords.length === 0 || keywords.some(k => original_data['keywords'][i].includes(k))) {

        filtered_abstract.push(original_data['abstract'][i]);
        filtered_web_url.push(original_data['web_url'][i]);
        filtered_headline.push(original_data['headline'][i]);
        filtered_section_name.push(original_data['section_name'][i]);
        filtered_keywords.push(original_data['keywords'][i]);
        filtered_pub_date.push(original_data['pub_date'][i]);
        filtered_sentiment.push(original_data['sentiment'][i]);
        //filtered_years.push(original_data['pub_date'][i]);
        if(original_data['sentiment'][i] == "Positive") {
          filtered_sent.push(1);
          filtered_years.push(original_data['pub_date'][i]);

        }
        else {
          filtered_sent.push(0);
          filtered_years.push(original_data['pub_date'][i]);

        }
      }
    }
  }


  for (let i = 0; i < filtered_years.length; i++) {
    let year = filtered_years[i];
    let value = filtered_sent[i];

  // If the year already exists in the object, increment the count of 1s
    if (value === 1) {
      if (yearCounts_pos[year] === undefined) {
        yearCounts_pos[year] = 1; // Initialize the count with 1 if not already in the object
      } else {
        yearCounts_pos[year] += 1; // Increment the count if the year already exists
    }
    }else{
      if (yearCounts_neg[year] === undefined) {
        yearCounts_neg[year] = 1; // Initialize the count with 1 if not already in the object
      } else {
        yearCounts_neg[year] += 1; // Increment the count if the year already exists
      }
    }
  }

  const keep_years = int_years.filter(year => year < active_year);


  keep_years.forEach(year => {
    if (yearCounts_pos[year] === undefined) {
      yearCounts_pos[year] = 0; // Add year with a frequency of 0 if not already in the object
    }
    if (yearCounts_neg[year] === undefined) {
      yearCounts_neg[year] = 0; // Add year with a frequency of 0 if not already in the object
    }
  });


  const year_pos = Object.keys(yearCounts_pos).map(year => parseInt(year));
  const year_neg = Object.keys(yearCounts_neg).map(year => parseInt(year));
  const pos_frequency = Object.values(yearCounts_pos);
  const neg_frequency = Object.values(yearCounts_neg);

  source.data = {
    abstract: filtered_abstract,
    web_url: filtered_web_url,
    headline: filtered_headline,
    section_name: filtered_section_name,
    keywords: filtered_keywords,
    pub_date: filtered_pub_date,
    sentiment: filtered_sentiment
  };

  display_source.data = {
    years_pos: year_pos,
    years_neg: year_neg,
    pos_frequency: pos_frequency,
    neg_frequency: neg_frequency,
  };

  pos_line.visible = checkbox_sentiment.active.includes(0);  // Turn on/off Positive line
  neg_line.visible = checkbox_sentiment.active.includes(1);
  pos_scat.visible = checkbox_sentiment.active.includes(0);  // Turn on/off Positive line
  neg_scat.visible = checkbox_sentiment.active.includes(1);

  source.change.emit();
  display_source.change.emit();

  keyword_display.text = keywords.join(', ');

""")


reset_button = Button(label = "Clear Keywords", button_type = "warning")

reset_callback = CustomJS(args=dict(source = source, filtered_source = filtered_source,keyword_display = keyword_display), code = """
    keyword_display.text = "";  // Clear the displayed keywords

    // Reset the filtered data
    filtered_source.data = {...source.data};

""")



year_slider.js_on_change('value', callback)
checkbox_section.js_on_change('active', callback)
checkbox_sentiment.js_on_change('active', callback)
keyword_search.js_on_change('value', callback)
reset_button.js_on_click(reset_callback) ##might bug out because it doesnt call callback also will fix

col1 = column(checkbox_section,checkbox_sentiment)
col2 = column(keyword_search,keyword_display,reset_button)
col3 = column(plot,year_slider)

layout = row(col1,col2,col3,log_div)

show(layout)


[2022, 2011, 2014, 2001, 2015, 2000, 2018, 2019, 2013, 2004, 2020, 2006, 2021, 2008, 2002, 2007, 2005, 2016, 2017, 2010, 2009, 2003, 2012, 2023]
[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
[20, 26, 14, 19, 23, 22, 29, 25, 22, 22, 20, 32, 16, 18, 22, 24, 18, 14, 14, 8, 20, 16, 22, 3]
[21, 30, 38, 34, 35, 22, 25, 21, 24, 18, 17, 24, 25, 16, 22, 19, 20, 18, 27, 20, 11, 11, 23, 10]


In [ ]:
import json

In [ ]:
int_years = articles['pub_date'].unique().tolist()
print(int_years)
int_years.sort()
print(int_years)
sections = articles['section_name'].unique().tolist()

start_pos = [articles[(articles['pub_date'] == year) & (articles['sentiment'] == "Positive")].shape[0] for year in int_years]
print(start_pos)
start_neg = [articles[(articles['pub_date'] == year) & (articles['sentiment'] == "Negative")].shape[0] for year in int_years]
print(start_neg)
total_start = [a + b for a, b in zip(start_pos, start_neg)]

#assign datasource
data = {
   'abstract' : articles['abstract'],
   'web_url' : articles['web_url'],
   'headline' : articles['headline'],
   'section' : articles['section_name'],
   'keywords': articles['keywords'],
   'pub_date': articles['pub_date'],
   'sentiment': articles['sentiment']
}

display_data = {
   'years_pos' : int_years,
   'years_neg': int_years,
   'pos_frequency': start_pos,
   'neg_frequency':  start_neg,
   'total_frequency': total_start,
   'display_freq': total_start
}

data = pd.DataFrame(data)
display_data = pd.DataFrame(data)

# Convert DataFrame to JSON
data_source = data.to_json(orient='records')
display_source = display_data.to_json(orient='records')

# Generate HTML with embedded JSON
d3_html_code = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <script src="https://d3js.org/d3.v7.min.js"></script>
    <title>D3 Interactive Plot</title>
    <style>
        .line {{ fill: none; stroke-width: 2px; }}
        .positive {{ stroke: green; }}
        .negative {{ stroke: red; }}
        .dot {{ stroke-width: 1px; fill-opacity: 0.6; }}
        .positive-dot {{ fill: green; }}
        .negative-dot {{ fill: red; }}
        #log_div {{ background-color: #f0f0f0; padding: 10px; border: 1px solid #ccc; height: 150px; overflow-y: scroll; }}
    </style>
</head>
<body>

    <div id="plotContainer" style="display: flex; justify-content: space-between;">
        <!-- Plot Area -->
        <svg id="plot" width="800" height="400"></svg>

        <!-- Display Box on the Right -->
        <div id="displayBox" style="width: 300px; height: 400px; overflow-y: scroll; border-left: 1px solid #ccc; padding: 10px;">
            <strong>Articles:</strong>
            <div id="articleList"></div>
        </div>
    </div>

    <div id="log_div">
        <strong>Debug Log:</strong>
        <p id="log_messages"></p>
    </div>
    <div>
        <label>Year:</label>
        <input type="range" id="yearSlider" min="2000" max="2025" value="2000">
        <span id="yearLabel">2000</span>
    </div>
    <div>
        <label>Sections:</label>
        <div id="sections"></div>
    </div>
    <div>
        <label>Sentiments:</label>
        <div id="sentiments">
            <input type="checkbox" id="positive" value="Positive" checked> Positive
            <input type="checkbox" id="negative" value="Negative" checked> Negative
        </div>
    </div>
    <div>
        <label>Keyword Search:</label>
        <input type="text" id="keywordInput">
    </div>
    <svg id="plot" width="800" height="400"></svg>

    <script>
        function updateArticleDisplay() {{
          updateLog("ARTICLE DISPLAY UPDATE CALLED")
          const keyword = d3.select("#keywordInput").property("value").toLowerCase();
          const selectedSections = Array.from(document.querySelectorAll("#sections input:checked")).map(d => d.value);
          const selectedSentiments = Array.from(document.querySelectorAll("#sentiments input:checked")).map(d => d.value);


          updateLog("ART1:");
          // Filter data based on selected criteria
          const filteredData = data.filter(d =>
              (selectedSections.includes("All") || selectedSections.includes(d.section)) &&
              selectedSentiments.includes(d.sentiment) &&
              (!keyword || d.keywords.includes(keyword))  // Filter by keyword
          );

          //updateLog(typeof filteredData[0].pub_date);

          updateLog("ART2:");


          // Group articles by year
          const articlesByYear = d3.group(filteredData, d => {{
              // Ensure pub_date is converted to a string
              const year = typeof d.pub_date === 'number'
                  ? d.pub_date.toString()
                  : String(d.pub_date);
              return year;
          }});


          // Convert Map to object for easier logging
          const articlesByYearObj = {{}};
          articlesByYear.forEach((value, key) => {{
              articlesByYearObj[key] = value;
          }});

          updateLog("ART3:");

          const articleListDiv = d3.select("#articleList");
          articleListDiv.html("");  // Clear previous entries

          updateLog("ART4:");

          // Iterate over each year group and create HTML structure
          Object.entries(articlesByYearObj).forEach(([year, articles]) => {{
              updateLog("IN ART YEAR1");
              const yearDiv = articleListDiv.append("div").attr("class", "yearGroup");
              updateLog("IN ART YEAR2");
              yearDiv.append("strong").text(year);  // Display the year
              updateLog("IN ART YEAR4");
              updateLog("YEAR GROUP " + JSON.stringify(articles));

              articles.forEach(article => {{
                  updateLog("IN ART YEAR5");
                  const articleDiv = yearDiv.append("div").attr("class", "article");
                  updateLog("IN ART YEAR6");
                  articleDiv
                      .html(`&#8226; ${{article.headline}}`)
                      .style("color", article.sentiment === "Positive" ? "green" : "red")
                      .style("cursor", "pointer")
                      .on("mouseover", function() {{
                          // Show the URL when hovering over the title
                          d3.select(this).append("span").text(" - " + article.web_url).style("color", "blue");
                          updateLog("MOUSE OVER");
                      }})
                      .on("mouseout", function() {{
                          // Remove the URL when not hovering
                          d3.select(this).select("span").remove();
                          updateLog("MOUSE OUT");
                      }})
                      .on("click", function() {{
                          // Open the URL when the title is clicked
                          window.open(article.web_url, "_blank");
                          updateLog("CLICK");
                      }});
                  updateLog("IN ART YEAR7");
              }});
          }});

          updateLog("ART5:");
          }}


        function updateLog(message) {{
            const logMessages = document.getElementById("log_messages");
            logMessages.innerHTML += message + "<br>";
            logMessages.scrollTop = logMessages.scrollHeight;  // Scroll to bottom
        }}

        updateLog("Initial Log");

        // Sample data: Replace with your datasets
        const data = {data_source};

        const displayData = {display_source};

        updateLog("HERE1");
        const firstRow = data[0];  // Get the first row

        // Set up plot dimensions
        const svg = d3.select("#plot");
        const margin = {{ top: 20, right: 30, bottom: 30, left: 40 }};
        const width = svg.attr("width") - margin.left - margin.right;
        const height = svg.attr("height") - margin.top - margin.bottom;
        const plotArea = svg.append("g").attr("transform", `translate(${{margin.left}},${{margin.top}})`);

        // Create a legend
        const legendWidth = 200;
        const legendHeight = 40;
        const legendSpacing = 20;  // Space between each item in the legend

        // Add a group for the legend
        const legend = svg.append("g")
            .attr("transform", `translate(${{width - margin.right}}, 20)`);  // Position the legend

        // Add a box for the positive sentiment
        legend.append("rect")
            .attr("x", 0)
            .attr("y", 0)
            .attr("width", 20)
            .attr("height", 20)
            .attr("fill", "green");  // Positive sentiment color

        // Add text for positive sentiment
        legend.append("text")
            .attr("x", 25)
            .attr("y", 15)
            .style("font-size", "12px")
            .text("Positive");

        // Add a box for the negative sentiment
        legend.append("rect")
            .attr("x", 0)
            .attr("y", legendSpacing)  // Add some space between the positive and negative boxes
            .attr("width", 20)
            .attr("height", 20)
            .attr("fill", "red");  // Negative sentiment color

        // Add text for negative sentiment
        legend.append("text")
            .attr("x", 25)
            .attr("y", legendSpacing + 15)
            .style("font-size", "12px")
            .text("Negative");


        svg.append("text")
          .attr("x", width / 2)  // Center the title horizontally
          .attr("y", margin.top - 10)  // Position the title above the plot
          .style("text-anchor", "middle")  // Center the text
          .style("font-size", "16px")  // Set the font size
          .style("font-weight", "bold")  // Make the title bold
          .text("My Plot Title");  // Set the title text

        updateLog("HERE2");

        let paddedMaxFreq = 50;

        let xScale = d3.scaleLinear().domain([2000, 2023]).range([0, width]);  // Ensure correct domain and range
        let yScale = d3.scaleLinear().domain([0, paddedMaxFreq]).range([height, 0]);


        updateLog("HERE3");

        //xScale.domain([2000, 2023]);
       // yScale.domain([0, paddedMaxFreq]); // Adjust based on your data

        updateLog("HERE4");

        // Add axes
        // Add gridlines for x-axis
  // Add gridlines for x-axis
        plotArea.append("g")
            .attr("class", "grid")
            .attr("transform", `translate(0,${{height}})`) // Position at the bottom of the plot area
            .call(d3.axisBottom(xScale)
                .ticks(xScale.domain()[1] - xScale.domain()[0] + 1) // Adjust for 1-unit spacing
                .tickSize(-height) // Length of the grid lines
                .tickFormat("")) // Hide tick labels
            .selectAll(".tick line")
            .attr("stroke", "gray") // Color of the gridlines
            .attr("stroke-opacity", 0.6) // Set opacity to 0.6
            .attr("shape-rendering", "crispEdges"); // For sharp grid lines

        // Add gridlines for y-axis
        plotArea.append("g")
            .attr("class", "grid")
            .call(d3.axisLeft(yScale)
                .ticks(5) // Adjust the number of ticks for y-axis
                .tickSize(-width) // Length of the grid lines
                .tickFormat("")) // Hide tick labels
            .selectAll(".tick line")
            .attr("stroke", "gray") // Color of the gridlines
            .attr("stroke-opacity", 0.6) // Set opacity to 0.6
            .attr("shape-rendering", "crispEdges"); // For sharp grid lines

        // Add the x-axis ticks (after gridlines to keep them on top)
        plotArea.append("g")
          .attr("class", "x-axis")
          .attr("transform", `translate(0, ${{height}})`)
          .call(d3.axisBottom(xScale));

        plotArea.append("g")
          .attr("class", "y-axis")
          .call(d3.axisLeft(yScale));

        plotArea.append("text")
            .attr("transform", `translate(${{width / 2}}, ${{height + margin.bottom - 10}})`)  // Position the x-axis label
            .style("text-anchor", "middle")
            .text("Year");

        plotArea.append("text")
            .attr("transform", `translate(-20, ${{height / 2}}) rotate(-90)`)  // Position the y-axis label vertically
            .style("text-anchor", "middle")
            .text("Number of Articles");

        updateLog("HERE5");

        const lineGenerator = d3.line()
            .x(d => {{
                const scaledX = xScale(d.year);
                return scaledX;
            }})
            .y(d => {{
                const scaledY = yScale(d.frequency);
                return scaledY;
            }});

        updateLog("HERE6");

        // Render the plot
        function updatePlot() {{
            const year = +d3.select("#yearSlider").property("value");
            const selectedSections = Array.from(document.querySelectorAll("#sections input:checked")).map(d => d.value);
            const selectedSentiments = Array.from(document.querySelectorAll("#sentiments input:checked")).map(d => d.value);
            const keyword = d3.select("#keywordInput").property("value").toLowerCase();


            // Filter data
            const filteredData = data.filter(d =>
                d.pub_date <= year &&
                (selectedSections.includes("All")||selectedSections.includes(d.section)) &&
                selectedSentiments.includes(d.sentiment) &&
                (!keyword || d.keywords.includes(keyword))
            );

            updateLog("HERE7");

            // Aggregate data for display
            const yearCountsPos = {{}};
            const yearCountsNeg = {{}};
            const years = Array.from({{ length: 2024 - 2000 + 1 }}, (_, i) => 2000 + i);

            years.forEach(y => {{
              if(y <= year){{
                yearCountsPos[y] = 0;
                yearCountsNeg[y] = 0;
              }}
            }});

            updateLog("HERE8");

            filteredData.forEach(d => {{

                if (d.sentiment === "Positive") {{
                    yearCountsPos[d.pub_date] = (yearCountsPos[d.pub_date] || 0) + 1;
                }} else if (d.sentiment === "Negative") {{
                    yearCountsNeg[d.pub_date] = (yearCountsNeg[d.pub_date] || 0) + 1;
                }}
            }});


            updateLog("HERE9");

            displayData.years_pos = Object.keys(yearCountsPos).map(Number);
            displayData.pos_frequency = Object.values(yearCountsPos);
            displayData.years_neg = Object.keys(yearCountsNeg).map(Number);
            displayData.neg_frequency = Object.values(yearCountsNeg);


            const maxPosFrequency = Math.max(...displayData.pos_frequency);
            const maxNegFrequency = Math.max(...displayData.neg_frequency);
            const maxFrequency = Math.max(maxPosFrequency, maxNegFrequency);
            paddedMaxFreq = maxFrequency + 2;

            xScale = d3.scaleLinear()
                .domain([2000, year])
                .range([0, width]);

            yScale = d3.scaleLinear()
                .domain([0, paddedMaxFreq])
                .range([height, 0]);

            updateLog("YDOMAIN : "+yScale.domain());

            updateLog("HERE10");
            plotArea.selectAll(".grid").remove();
            plotArea.selectAll(".x-axis").remove();
            plotArea.selectAll(".y-axis").remove();

            // Re-add gridlines for the x-axis
            plotArea.append("g")
                .attr("class", "grid")
                .attr("transform", `translate(0,${{height}})`) // Position at the bottom of the plot area
                .call(d3.axisBottom(xScale)
                    .ticks(xScale.domain()[1] - xScale.domain()[0]) // Adjust for 1-unit spacing
                    .tickSize(-height) // Length of the grid lines
                    .tickFormat("")) // Hide tick labels
                .selectAll(".tick line")
                .attr("stroke", "gray") // Color of the gridlines
                .attr("stroke-opacity", 0.6) // Set opacity to 0.6
                .attr("shape-rendering", "crispEdges"); // For sharp grid lines

            // Re-add gridlines for the y-axis
            plotArea.append("g")
                .attr("class", "grid")
                .call(d3.axisLeft(yScale)
                    .ticks(5) // Adjust the number of ticks for y-axis
                    .tickSize(-width) // Length of the grid lines
                    .tickFormat("")) // Hide tick labels
                .selectAll(".tick line")
                .attr("stroke", "gray") // Color of the gridlines
                .attr("stroke-opacity", 0.6) // Set opacity to 0.6
                .attr("shape-rendering", "crispEdges"); // For sharp grid lines
            // Bind and update lines and dots
            const positiveData = displayData.years_pos.map((year, i) => ({{ year, frequency: displayData.pos_frequency[i] }}));
            const negativeData = displayData.years_neg.map((year, i) => ({{ year, frequency: displayData.neg_frequency[i] }}));

            updateLog("HERE11");


            updateLog("HERE11.1");

            //.attr("transform", `translate(0,${{height}})`)

                //.attr("transform", `translate(0,${{height}})`)

            plotArea.append("g")
                .attr("class", "x-axis")
                .attr("transform", `translate(0, ${{height}})`)
                .transition()
                .duration(500)
                .call(d3.axisBottom(xScale).tickFormat(d3.format("d")));

            updateLog("HERE11.1  X AXIS DONE");

            const tickCount = (paddedMaxFreq / 100) % 10 + 10;
            plotArea.append("g")
                .attr("class", "y-axis")
                .transition()
                .duration(500)
                .call(d3.axisLeft(yScale).ticks(tickCount));

            updateLog("HERE BEFORE PLOT");
            plotArea.selectAll(".line.positive")
                .data([positiveData])  // Ensure we bind the data correctly to the line
                .join("path")
                .attr("class", "line positive")
                .transition() // Add a transition for smooth updates
                .duration(500)
                .attr("d", lineGenerator)

            plotArea.selectAll(".line.negative")
                .data([negativeData])  // Ensure we bind the data correctly to the line
                .join("path")
                .attr("class", "line negative")
                .transition() // Add a transition for smooth updates
                .duration(500)
                .attr("d", lineGenerator)

            updateLog("HERE11.3");

            //plotArea.selectAll(".y-axis")
              //  .transition() // Add a transition for smooth updates
                //.duration(500)
                //.call(d3.axisLeft(yScale)
                 //   .tickFormat(d3.format("d")));  // Update the y-axis
        }}

        // Initialize UI components
        const uniqueSections = Array.from(new Set(data.map(d => d.section)));
       // Initialize UI components
      const sectionsContainer = d3.select("#sections");

      // Add "All" checkbox first
      sectionsContainer.append("label")
          .html('<input type="checkbox" id="allSectionsCheckbox" value="All"> All');

      // Add section checkboxes
      sectionsContainer.selectAll(".section-label")
          .data(uniqueSections)
          .enter()
          .append("label")
          .attr("class", "section-label")
          .html(d => `<input type="checkbox" class="section-checkbox" value="${{d}}"> ${{d}}`);

      // Event listener
      document.getElementById('allSectionsCheckbox').addEventListener('change', function() {{
          updateLog("ALL SECTION CHANGE");
          const isChecked = this.checked;
          updateLog("CHANGE TO : " + isChecked);

          // Get all section checkboxes
          const sectionCheckboxes = document.querySelectorAll('.section-checkbox');

          // Enable or disable other checkboxes based on "All" selection
          if(isChecked){{
              sectionCheckboxes.forEach(checkbox => {{
                  checkbox.checked = false;
                  checkbox.disabled = true;
              }});
          }} else {{
              sectionCheckboxes.forEach(checkbox => {{
                  checkbox.disabled = false;
              }});
          }}

          // Trigger plot update
          updatePlot();
      }});
        //d3.selectAll(".section-checkbox").on("change", function () {{
          //const allChecked = d3.selectAll(".section-checkbox").nodes().every(node => node.checked);

          // Uncheck "All" if not all sections are selected
          //d3.select("#allSections").property("checked", allChecked === true);

          // Trigger plot update
         // updatePlot();
      //}});

        updateLog("HERE13");
        updateLog("Unique sections: " + uniqueSections.join(", "));
        // Attach event listeners
        d3.select("#yearSlider").on("input", function () {{
            d3.select("#yearLabel").text(this.value);
            updatePlot();
        }});

        updateLog("HERE14");
        d3.selectAll("#sections input, #sentiments input").on("change", updatePlot);
        d3.select("#keywordInput").on("keydown", function(event) {{
            if (event.key === "Enter") {{ // Check if the key pressed is Enter
                updatePlot();
                updateArticleDisplay();
            }}
        }});

        updateLog("HERE15");

        // Initial plot render
        updatePlot();
    </script>
</body>
</html>

"""

# Save to a file
html_file_path = '/content/drive/MyDrive/Colab Notebooks/InfoVis_FinalProject/InfoVisFinalProject/d3_plot.html'
with open(html_file_path, 'w') as f:
    f.write(d3_html_code)

# Display the HTML in an iframe
from IPython.core.display import HTML

iframe_code = f"""
<iframe src="/content/drive/MyDrive/Colab Notebooks/InfoVis_FinalProject/InfoVisFinalProjectd3_plot.html" width="900" height="500" style="border:none;"></iframe>
"""
HTML(iframe_code)


[2022, 2011, 2014, 2001, 2015, 2000, 2018, 2019, 2013, 2004, 2020, 2006, 2021, 2008, 2002, 2007, 2005, 2016, 2017, 2010, 2009, 2003, 2012, 2023]
[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
[20, 26, 14, 19, 23, 22, 29, 25, 22, 22, 20, 32, 16, 18, 22, 24, 18, 14, 14, 8, 20, 16, 22, 3]
[21, 30, 38, 34, 35, 22, 25, 21, 24, 18, 17, 24, 25, 16, 22, 19, 20, 18, 27, 20, 11, 11, 23, 10]
